In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import os
from json import load

In [ ]:
CURRENT_DIR = Path(os.getcwd())
IMAGE_DIR = CURRENT_DIR / "little_pics"
picture_settings_fp = IMAGE_DIR / "picture_settings.json"
if picture_settings_fp.exists():
    SETTINGS_DICT = load(open(picture_settings_fp, "r"))
else:
    raise FileNotFoundError(f"File {picture_settings_fp} not found")


def get_fine_edges(image, blur_kernel=(15, 15), threshold=50):
    image_blurred = cv.GaussianBlur(image, blur_kernel, 0)
    image_fine_edges = cv.Canny(
        image_blurred, threshold1=threshold, threshold2=threshold * 2
    )
    return image_fine_edges


def get_contours(
    image,
    blur_kernel=(5, 5),
    grad_kernel=(35, 35),
    num_contours=1,
    arclength_epsilon=0.004,
):
    image_blurred = cv.GaussianBlur(image, blur_kernel, 0)
    grad_kernel_mat = np.ones(grad_kernel, np.uint8)
    morph_grad = cv.morphologyEx(image_blurred, cv.MORPH_GRADIENT, grad_kernel_mat)
    _, image_thresh = cv.threshold(
        morph_grad, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU
    )
    contours, _ = cv.findContours(
        image_thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE
    )
    contours_sorted = sorted(contours, key=cv.contourArea, reverse=True)[:num_contours]
    approximated_contours = []
    for contour in contours_sorted:
        epsilon = arclength_epsilon * cv.arcLength(contour, True)
        approx_contour = cv.approxPolyDP(contour, epsilon, True)
        approximated_contours.append(np.squeeze(approx_contour))

    return approximated_contours


def process_images(
    settings_dict: dict = None, images_dir: Path | str = IMAGE_DIR
) -> dict:

    if images_dir.exists():
        images_fns = images_dir.iterdir()
    else:
        raise FileNotFoundError(f"Directory {images_dir} not found")

    if settings_dict is None:
        settings_dict = {fn.name: {"format": fn.suffix} for fn in images_fns}

    processed_images = {}
    for key, settings in settings_dict.items():
        image_fn = images_dir / f"{key}.{settings['format']}"
        image = cv.imread(str(image_fn), cv.IMREAD_GRAYSCALE)
        if image is None:
            raise ValueError(f"Image {image_fn.name} not found in {images_dir}")

        if "fine_blur_kernel" in settings.keys():
            fine_blur_kernel = settings["fine_blur_kernel"]
        else:
            fine_blur_kernel = (15, 15)

        if "contour_blur_kernel" in settings.keys():
            contour_blur_kernel = settings["contour_blur_kernel"]
        else:
            contour_blur_kernel = (3, 3)

        if "grad_kernel" in settings.keys():
            grad_kernel = settings["grad_kernel"]
        else:
            grad_kernel = (55, 55)

        if "threshold" in settings.keys():
            threshold = settings["threshold"]
        else:
            threshold = 10

        if "num_contours" in settings.keys():
            num_contours = settings["num_contours"]
        else:
            num_contours = 1

        if "arclength_epsilon" in settings.keys():
            arclength_epsilon = settings["arclength_epsilon"]
        else:
            arclength_epsilon = 0.004

        processed_images[key] = {
            "image": image,
            "fine_edges": get_fine_edges(
                image, blur_kernel=fine_blur_kernel, threshold=threshold
            ),
            "contours": get_contours(
                image,
                blur_kernel=contour_blur_kernel,
                grad_kernel=grad_kernel,
                num_contours=num_contours,
                arclength_epsilon=arclength_epsilon,
            ),
        }

    return processed_images


def plot_images(images_dict: dict):
    figheight = 3 * len(images_dict)
    figwidth = 9
    fig, axs = plt.subplots(
        len(images_dict), 3, figsize=(figwidth, figheight), squeeze=True
    )
    axs = axs.flatten()
    for i, key in enumerate(images_dict.keys()):
        image = images_dict[key]["image"]
        fine_edges = images_dict[key]["fine_edges"]
        contours = images_dict[key]["contours"]
        axs[i * 3].axis("off")
        axs[i * 3].imshow(image, cmap="gray")
        axs[i * 3 + 1].axis("off")
        axs[i * 3 + 1].imshow(fine_edges, cmap="gray")
        axs[i * 3 + 2].axis("off")
        axs[i * 3 + 2].imshow(fine_edges, cmap="gray", zorder=0)
        for contour in contours:
            axs[i * 3 + 2].plot(contour[:, 0], contour[:, 1], zorder=1)

    return fig


CONTOURS_DIR = CURRENT_DIR / "contours"


def save_contours(images_dict: dict, save_dir: Path | str = CONTOURS_DIR):
    for key in images_dict.keys():
        subfolder = save_dir / key
        if not subfolder.exists():
            subfolder.mkdir(parents=True)
        for file in subfolder.iterdir():
            file.unlink()
        contours = images_dict[key]["contours"]
        for idx, contour in enumerate(contours):
            np.savetxt(subfolder / f"{idx}.txt", contour)


EDGES_DIR = CURRENT_DIR / "edges"


def save_edges(images_dict: dict, save_dir: Path | str = EDGES_DIR):
    if not save_dir.exists():
        save_dir.mkdir(parents=True)
    for key in images_dict.keys():
        fine_edges = images_dict[key]["fine_edges"]
        size = fine_edges.shape
        dpi = 1000
        figsize = size[1] / dpi, size[0] / dpi
        fig, ax = plt.subplots(1, 1, figsize=figsize)
        ax.imshow(fine_edges, cmap="gray")
        ax.axis("off")
        fp = save_dir / f"{key}.png"
        np.savetxt(save_dir / f"{key}.txt", [size])
        print(f"Saved image {key} with size {size} to {fp}")
        fig.savefig(fp, bbox_inches="tight", pad_inches=0, dpi=dpi, transparent=True)
        plt.close(fig)

In [ ]:
images_dict = process_images(SETTINGS_DICT)
fig = plot_images(images_dict)
fig.tight_layout()
plt.show(fig)

In [ ]:
save_contours(images_dict)
save_edges(images_dict)